# Worksheet 08

Name: Jonathan Thomas
UID: U81823882

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [8]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.032223486743153, 5.525617416397002, 3.9486963459444384, 5.16547214840305, 6.080129784977727, 6.647717147233947, 5.359126750755612, 4.545244339836287, 5.002858905928765, 4.1520204035864]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [9]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.8633266849454335, 8.389713079598563, 6.213519970896894, 6.581015714106089, 8.13854609130395, 8.285021868738957, 5.769393431820069, 9.669345889006534, 9.658640617136228, 8.868827671590864]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [10]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.1520204035864, 5.002858905928765, 8.868827671590864, 4.545244339836287, 9.658640617136228, 9.669345889006534, 5.769393431820069, 5.359126750755612, 6.647717147233947, 8.285021868738957]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Means and Variances of each species distribution (s1 ~ Normal(10, 1) & s2 ~ Normal(8,1))
- Prior probabilites of each species (coin flip/0.5 for both)


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [57]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

print(data, kmeans.labels_)
s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.1520204035864, 5.002858905928765, 8.868827671590864, 4.545244339836287, 9.658640617136228, 9.669345889006534, 5.769393431820069, 5.359126750755612, 6.647717147233947, 8.285021868738957] [0 0 1 0 1 1 0 0 0 1]
[4.1520204035864, 5.002858905928765, 4.545244339836287, 5.769393431820069, 5.359126750755612, 6.647717147233947]
[8.868827671590864, 9.658640617136228, 9.669345889006534, 8.285021868738957]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.246060163193513,  mean_2 = 9.120459011618147
var_1 = 0.6664194549313046,  var_2 = 0.3380474489183162


Close

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [58]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

print(mean[0], mean[1])
for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ",pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


5.246060163193513 9.120459011618147
point =  4.1520204035864
probability of observing that point if it came from cluster 0 =  0.19908054523228497
probability of observing that point if it came from cluster 1 =  9.540830563796795e-17
point =  5.002858905928765
probability of observing that point if it came from cluster 0 =  0.4674807394040476
probability of observing that point if it came from cluster 1 =  8.820450384888993e-12
point =  8.868827671590864
probability of observing that point if it came from cluster 0 =  2.5855032731028925e-05
probability of observing that point if it came from cluster 1 =  0.6248103904015908
point =  4.545244339836287
probability of observing that point if it came from cluster 0 =  0.3380659468059633
probability of observing that point if it came from cluster 1 =  2.4559529436770045e-14
point =  9.658640617136228
probability of observing that point if it came from cluster 0 =  2.2111401873766148e-07
probability of observing that point if it came from clus

I think every point is accurately labeled here; points closer to 5 are set to 0 and those close to 8 are set to 1

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [59]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * ((x[1] - mean[0]) ** 2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * ((x[1] - mean[1]) ** 2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 5.247468233932487,  mean_2 = 9.12077600408126
var_1 = 0.6706258992089454,  var_2 = 0.3384855867933154


Very close to the orignal.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [60]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))
        print("probability of observing that point if it came from cluster " + str(j) + " = ",pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.1520204035864
probability of observing that point if it came from cluster 0 =  0.19911800344926153
probability of observing that point if it came from cluster 1 =  9.949684168416914e-17
point =  5.002858905928765
probability of observing that point if it came from cluster 0 =  0.4659033922263087
probability of observing that point if it came from cluster 1 =  9.070516679454789e-12
point =  8.868827671590864
probability of observing that point if it came from cluster 0 =  2.7625205091386437e-05
probability of observing that point if it came from cluster 1 =  0.6243343419731926
point =  4.545244339836287
probability of observing that point if it came from cluster 0 =  0.33728669969131614
probability of observing that point if it came from cluster 1 =  2.5437987223718933e-14
point =  9.658640617136228
probability of observing that point if it came from cluster 0 =  2.4381914557595375e-07
probability of observing that point if it came from cluster 1 =  0.44724719118800677
point 

Largely unchanged.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [71]:

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    index = (p[1], p[2]).index(max(p[1], p[2]))
    print("Point",p[0],":",index)

Point 4.1520204035864 : 0
Point 5.002858905928765 : 0
Point 8.868827671590864 : 1
Point 4.545244339836287 : 0
Point 9.658640617136228 : 1
Point 9.669345889006534 : 1
Point 5.769393431820069 : 0
Point 5.359126750755612 : 0
Point 6.647717147233947 : 0
Point 8.285021868738957 : 1
